In [65]:
import requests
import passwords as pw
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [33]:
def get_puuid(game_name, tag_line, api_key):
    api_url = (f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}?api_key={api_key}")
    resp = requests.get(api_url)
    player_info = resp.json()
    return player_info['puuid']


In [34]:
def get_active_game(puuid, api_key):
    headers = {"X-Riot-Token": api_key}
    api_url = (f"https://na1.api.riotgames.com/lol/spectator/v5/active-games/by-summoner/{puuid}")
    resp = requests.get(api_url, headers = headers)
    game_data = resp.json()

    champion_ids = [player['championId'] for player in game_data['participants']]
    return champion_ids


In [35]:
def champion_names(champ_ids):
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"
    champs = requests.get(api_url).json()["data"]
    id_to_name = {int(name["key"]): champ for champ, name in champs.items()}
    
    champ_names = [id_to_name.get(id, "Unknown") for id in champ_ids]
    print("Champions in game:", champ_names)

In [36]:
def get_match_ids(puuid, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" +
        puuid + "/ids?start=0&count=10&api_key=" + api_key
    )
    resp = requests.get(api_url)
    match_ids = resp.json()
    return match_ids

In [37]:
def get_match_data(match_id, api_key):
    api_url = ("https://americas.api.riotgames.com/lol/match/v5/matches/" + match_id +
               "?api_key=" + api_key)
    try:
        resp = requests.get(api_url)
        match_data = resp.json()
        return match_data
    except:
        return None

In [38]:
puuid = get_puuid("McDonaldsManager", "mid", pw.API_KEY)
puuid

'bCnPTf3VMxhRVLuPR3147HhV_hUrIRYSFt4fWgFUcIivP9tFWcJfuRWLwIliB8qChi9cIvcLztpaFA'

In [71]:
dataframe = []
match_ids = get_match_ids(pw.dl_PUUID, pw.API_KEY)
for match in match_ids:
    match_data = get_match_data(match, pw.API_KEY)
    if not match_data:
        continue

    participants = match_data['info']['participants']
    ally_champs = [p['championName'] for p in participants if p['teamId'] == 100]
    enemy_champs = [p['championName'] for p in participants if p['teamId'] == 200]
    
    dataframe.append({
        'match_id': match,
        'ally_champs': ally_champs,
        'enemy_champs': enemy_champs
    })
    


In [72]:
df = pd.DataFrame(dataframe)
df

,match_id,ally_champs,enemy_champs
0,NA1_5311257006,"[Yone, Talon, Syndra, Jhin, TwistedFate]","[Riven, LeeSin, Sylas, Seraphine, Janna]"
1,NA1_5311227228,"[Gwen, LeeSin, Akali, MissFortune, Nautilus]","[Sett, Rengar, TwistedFate, Tristana, Yuumi]"
2,NA1_5311203254,"[Volibear, Udyr, TwistedFate, Ezreal, Soraka]","[Gwen, Zed, Yone, Lux, Ashe]"
3,NA1_5311170342,"[Kindred, Zac, Galio, Varus, Elise]","[Ambessa, Viego, TwistedFate, Ekko, Nautilus]"
4,NA1_5311141585,"[Malphite, MasterYi, TwistedFate, Tristana, Le...","[Camille, Viego, Zed, Malzahar, Renata]"
5,NA1_5311116625,"[Renekton, Shaco, Taliyah, Jhin, Thresh]","[Camille, Hecarim, TwistedFate, Ezreal, Nami]"
6,NA1_5311093043,"[Sylas, LeeSin, Kayle, Lucian, JarvanIV]","[Vladimir, Hecarim, TwistedFate, Senna, Alistar]"
7,NA1_5311069610,"[Irelia, Yorick, TwistedFate, Vayne, Milio]","[Chogath, Ekko, Yasuo, MissFortune, Nidalee]"
8,NA1_5311049753,"[DrMundo, Elise, Nunu, Jhin, Pyke]","[Renekton, Sejuani, Vladimir, Akshan, Morgana]"
9,NA1_5311023309,"[Singed, Nocturne, Viktor, Jhin, Brand]","[Renekton, DrMundo, Xerath, Sivir, Alistar]"


In [ ]:
def get_all_champs():
    version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
    api_url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json"

    champs_url = f'https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion.json'
    resp = requests.get(champs_url).json()

    champ_data = resp['data']
    champ_names = list(champ_data.keys())

    return champ_names

['Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Ambessa',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'AurelionSol',
 'Aurora',
 'Azir',
 'Bard',
 'Belveth',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Briar',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'Draven',
 'DrMundo',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Hwei',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 'Khazix',
 'Kindred',
 'Kled',
 'KogMaw',
 'KSante',
 'Leblanc',
 'LeeSin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'MasterYi',
 'Mel',
 'Milio',
 'MissFortune',
 'MonkeyKing',
 'Mordekaiser',
 'Morgana',
 'Naafiri',
 'Nami',
 'Nasu

In [79]:
#One Hot Encoding

all_champs = get_all_champs()

mlb_allies = MultiLabelBinarizer(classes=all_champs)
mlb_enemies = MultiLabelBinarizer(classes=all_champs)

ally_encoded = pd.DataFrame(mlb_allies.fit_transform(df['ally_champs']),
                            columns=[f'ally_{c}' for c in mlb_allies.classes_],
                            index=df.index)

enemy_encoded = pd.DataFrame(mlb_enemies.fit_transform(df['enemy_champs']),
                             columns=[f'enemy_{c}' for c in mlb_enemies.classes_],
                             index=df.index)

final_df = pd.concat([df[['match_id']], ally_encoded, enemy_encoded], axis=1)

final_df


,match_id,ally_Aatrox,ally_Ahri,ally_Akali,ally_Akshan,ally_Alistar,ally_Ambessa,ally_Amumu,ally_Anivia,ally_Annie,...,enemy_Yone,enemy_Yorick,enemy_Yuumi,enemy_Zac,enemy_Zed,enemy_Zeri,enemy_Ziggs,enemy_Zilean,enemy_Zoe,enemy_Zyra
0,NA1_5311257006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NA1_5311227228,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,NA1_5311203254,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,NA1_5311170342,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NA1_5311141585,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,NA1_5311116625,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,NA1_5311093043,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,NA1_5311069610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,NA1_5311049753,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,NA1_5311023309,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
df.to_csv('df.csv', index = False)